In [9]:
%pip install tensorflow==2.15.0 pandas scikit-learn matplotlib tqdm
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

def evaluate_model(model_path):
    """
    Evaluates a single model and returns its metrics
    """
    print(f"\nEvaluating model: {model_path}")
    print("="*50)
    
    if not os.path.exists(model_path):
        print(f"Error: Model file {model_path} does not exist!")
        return None
    
    # Load the model
    try:
        # Use TensorFlow's implementation directly
        model = tf.keras.models.load_model(model_path, compile=False)
        model.compile(run_eagerly=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        print("Full error details:", e)
        return None
    
    # Constants
    PAIRS_FILE_PATH = "11-785-fall-20-homework-2-part-2/verification_pairs_val.txt"
    DATA_ROOT = "./"
    IMG_HEIGHT = 160  # Changed to match the model's expected input size
    IMG_WIDTH = 160   # Changed to match the model's expected input size
    
    def preprocess_image(image_path):
        try:
            full_path = os.path.join(DATA_ROOT, image_path)
            if not os.path.exists(full_path):
                print(f"Warning: Image file does not exist: {full_path}")
                return None
                
            img = tf.io.read_file(full_path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
            img = tf.expand_dims(img, axis=0)
            # Simple normalization to [0,1] range
            img = img / 255.0
            return img
        except Exception as e:
            print(f"Warning: Could not load image {image_path}. Error: {e}")
            return None

    def get_embedding(image_path):
        processed_img = preprocess_image(image_path)
        if processed_img is None:
            return None
        try:
            embedding = model.predict(processed_img, verbose=0)
            return embedding[0]
        except Exception as e:
            print(f"Error generating embedding: {e}")
            return None

    def cosine_similarity(emb1, emb2):
        emb1_norm = emb1 / (np.linalg.norm(emb1) + 1e-10)
        emb2_norm = emb2 / (np.linalg.norm(emb2) + 1e-10)
        return np.dot(emb1_norm, emb2_norm)

    def euclidean_distance(emb1, emb2):
        return np.linalg.norm(emb1 - emb2)

    # Check if pairs file exists
    if not os.path.exists(PAIRS_FILE_PATH):
        print(f"Error: Pairs file {PAIRS_FILE_PATH} does not exist!")
        return None

    # Load and process pairs
    print(f"Loading verification pairs...")
    try:
        pairs_df = pd.read_csv(
            PAIRS_FILE_PATH,
            sep=' ',
            header=None,
            names=['img1_path', 'img2_path', 'label']
        )
    except Exception as e:
        print(f"Error loading pairs file: {e}")
        return None

    y_true = []
    y_scores_cosine = []
    y_scores_euclidean = []
    
    print(f"Generating embeddings for {len(pairs_df)} pairs...")

    for _, row in tqdm(pairs_df.iterrows(), total=len(pairs_df)):
        emb1 = get_embedding(row['img1_path'])
        emb2 = get_embedding(row['img2_path'])

        if emb1 is None or emb2 is None:
            continue

        cos_sim = cosine_similarity(emb1, emb2)
        euc_dist = euclidean_distance(emb1, emb2)
        
        y_true.append(row['label'])
        y_scores_cosine.append(cos_sim)
        y_scores_euclidean.append(-euc_dist)

    if not y_true:
        print("Error: No pairs were successfully processed.")
        return None

    # Calculate metrics
    results = {
        'model_name': os.path.basename(model_path).split('.')[0],
        'auc_cosine': roc_auc_score(y_true, y_scores_cosine),
        'auc_euclidean': roc_auc_score(y_true, y_scores_euclidean),
        'y_true': y_true,
        'y_scores_cosine': y_scores_cosine
    }
    
    return results

# List of models to evaluate
models = [
    "face_embedding_model.keras",
    "face_metric_embedder_batch_hard.keras"
]

# Verify models exist
for model_path in models:
    if not os.path.exists(model_path):
        print(f"Warning: Model file {model_path} does not exist in the current directory!")

# Evaluate all models
results = []
for model_path in models:
    result = evaluate_model(model_path)
    if result:
        results.append(result)

if not results:
    print("No models were successfully evaluated!")
else:
    # Plot comparison
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'red']
    linestyles = ['-', '--']

    for result, color, linestyle in zip(results, colors, linestyles):
        fpr, tpr, _ = roc_curve(result['y_true'], result['y_scores_cosine'])
        plt.plot(fpr, tpr, color=color, lw=2, linestyle=linestyle,
                 label=f"{result['model_name']} (AUC = {result['auc_cosine']:.4f})")

    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle=':')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('ROC Curve Comparison')
    plt.legend(loc="lower right")
    plt.grid(True)

    # Save the comparison plot
    plt.savefig("model_comparison_roc.png")
    print("\nModel Comparison Results:")
    print("="*50)
    for result in results:
        print(f"\nModel: {result['model_name']}")
        print(f"Cosine Similarity AUC:    {result['auc_cosine']:.4f}")
        print(f"Euclidean Distance AUC:    {result['auc_euclidean']:.4f}")
    print("\nComparison plot saved as 'model_comparison_roc.png'")

Note: you may need to restart the kernel to use updated packages.

Evaluating model: face_embedding_model.keras
Error loading model: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_1', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 160, 160, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'efficientnetb0', 'trainable': False, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 160, 160, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}, 'registered_name': None, 'name': 'input_layer_1', 'inbound_no